In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import io
import base64
import random

import gym
from gym import wrappers
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import torch

from lib import get_network
from lib.agent import DQN
from lib.logging import Logger
from lib.rollout import ReplayBuffer

from IPython.display import HTML

plt.style.use('seaborn')

### In this assignment you are going to:
* implement DQN and doubleDQN
* use them on CartPole, LunarLander and (Optionally) BreakOut environments

### We use PyTorch for neural networks. If you are new to PyTorch see tutorials:
https://pytorch.org/tutorials/ <br>
https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html

# CartPole

First, let's take a closer look at the environment.

In [3]:
def make_env(name, seed=None, monitor=False):
    env = gym.make(name)
    if monitor:
        env = wrappers.Monitor(env, "./gym-results", force=True)
    if seed:
        env.seed(seed)
    return env

In [4]:
def get_html_video_string(infix):
    video = io.open(
        f'./gym-results/openaigym.video.{infix}.video000000.mp4',
        'r+b').read()
    encoded = base64.b64encode(video)
    dec_str = encoded.decode('ascii')
    src_tag = f'<source src="data:video/mp4;base64,{dec_str}" type="video/mp4" /></video>'
    html = f'<video width="360" height="auto" alt="test" controls>{src_tag}</video>'
    return html

In [5]:
SEED = 0xCEEEEEEB
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

In [6]:
log_root = Path("results")
log_root.mkdir(exist_ok=True)

The next cell will embed a video of random policy playing CartPole-v1

In [7]:
env = make_env("CartPole-v0", monitor=True)
env.reset()
while True:
    action = env.action_space.sample()
    if env.step(action)[2]: break
env.close()

HTML(data=get_html_video_string(env.file_infix))

## Training loop
Here's the definition of a function that implements training loop. We'll get through its elements soon.
Although, note that we use a warmup techinque to fill the buffer in the beginning of a training procedure.

The loop consists of:
* Picking an action using the policy:
$$a = \text{argmax}_a Q(s, a)$$
* Sending the action to environment (`step`)
* Sampling N examples from the ReplayBuffer:
$$ (s_i, a_i, s^\prime_i, r_i)_{i=1,..,N} $$
* Updating Q-network by minimizing the loss:
$$y = r_i + \gamma \text{max}_a Q_{target}(s^{\prime}_i, a)$$

$$L_i = \text{Loss}(Q(s_i, a_i), y)$$
$$L = \frac{1}{N} \sum_i L_i \rightarrow min,$$
where $\text{Loss}$ is $L_2$ loss or Huber loss
* Updating target network every `target_update_every` iteration:
$$\theta^\prime \leftarrow \theta \tau + \theta^\prime (1 - \tau)$$
where $\theta^\prime$ - paremeters of $Q_{target}$ network, <br>
$\theta$ - paremeters of $Q$ network,<br>
$\tau \in [0, 1]$

Note, that we also use a logger that writes training info to `CSV` file, `TensorBoard` and `stdout`. We use `CSV` and `stdout` throughout the notebook, but you can use TB for convenience.

#### Task 0: Implement ReplayBuffer from `lib/rollout.py`

In [8]:
def train(
    agent,
    env,
    *,
    log_dir,
    prefix,
    buffer_size,
    n_steps,
    warmup_steps,
    target_update_every,
    log_every,
    save_every
):
    """Training loop procedure.
    
    Arguments:
        agent: DQN object.
        env: Gym environment.
        log_dir: PathLib path to logging dir.
        prefix: Prefix for the experiment.
        buffer_size: Maximum capacity of the replay buffer.
        n_steps: Total number of training iterations.
        warmup_steps: Number of first iterations that employ random policy.
        target_update_every: Number of iterations between target network updates.
        log_every: Log frequency
        save_every: Save frequency
        
    Returns agent after training.
    """
    
    logger = Logger(log_dir, prefix)
    episode_reward = []
    memory = ReplayBuffer(buffer_size)
    
    obs_cur = env.reset()
    for i in range(n_steps + 1):
        env.render()
        if i > warmup_steps:
            obs = torch.FloatTensor(obs_cur).unsqueeze(0)
            act = agent.pick_action(obs)
        else:
            act = env.action_space.sample()

        obs_prev = obs_cur
        obs_cur, rew, done, info = env.step(act)
        x, x_dot, theta, theta_dot = obs_cur
        r1 = (env.x_threshold - abs(x)) / env.x_threshold - 0.8
        r2 = (env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians - 0.5
        rew = r1 + r2
        episode_reward.append(rew)

        memory.push(obs_prev, act, obs_cur, rew, done)

        if done:
            rew = np.sum(episode_reward)
            episode_reward = []
            logger.log_arr_kv("reward/reward", rew)
            obs_cur = env.reset()
            logger.reduce_arr_kv("reward/reward", "reward/reward_mean", np.mean)
            logger.reduce_arr_kv("reward/reward", "reward/reward_max", np.max)
            logger.reduce_arr_kv("reward/reward", "reward/reward_min", np.min)
            logger.reduce_arr_kv("reward/reward", "reward/reward_std", np.std)
            logger.reduce_arr_kv("loss/bellman_error_t", "loss/bellman_error", np.mean)
            logger.reduce_arr_kv("misc/q_t", "misc/q", np.mean)
            logger.reduce_arr_kv("misc/q_est_t", "misc/q_est", np.mean)
            logger.log_kv("misc/epsilon", agent.eps)
            logger.log_kv("misc/timestep", i)
            '''
            logger.write_logs(skip_arrs=True)
            '''

        if i > warmup_steps:

            loss, q, q_est = agent.update_value(memory)
            logger.log_arr_kv("loss/bellman_error_t", loss)
            logger.log_arr_kv("misc/q_t", q)
            logger.log_arr_kv("misc/q_est_t", q_est)

            if i % target_update_every == 0:
                agent.update_target()

            if i % save_every == 0:
                num = i // save_every + 1
                save_dir = log_dir / "checkpoints"
                save_dir.mkdir(exist_ok=True)
                agent.save_to(save_dir, prefix=hex(num)[2:].upper())

        agent.update_eps()
    return agent

# Implementation
#### Task 1: Implement MLP network in `lib/network.py`
#### Task 2: Implement DQN in `lib/agent.py`

Now, run the next two cells. You should obtain the highest (500) reward in less than 100000 steps

Note! You can try to use small `target_update_every`. E.g. value of 1 corresponds to fitted Q-iteration. The expected outcome is unstable or even divergent behaviour

In [9]:
env = gym.make('CartPole-v0')
env = env.unwrapped

value_network = get_network("mlp")(
    env.observation_space.shape[0],
    64,
    env.action_space.n)

agent = DQN(
    value_network,
    env.action_space,
    eps=.1,
    eps_decay=0.,
    batch_size=32,
    learning_rate=0.01,
    discount_factor=.9
)

In [10]:
agent = train(
    agent,
    env,
    log_dir=log_root / "cartpole-v1",
    prefix="_dqn",
    buffer_size=2000,
    n_steps=10**5,
    warmup_steps=2000,
    target_update_every=100,
    log_every=500,
    save_every=1000,
)

KeyboardInterrupt: 

In [ ]:
"""
View more, visit my tutorial page: https://morvanzhou.github.io/tutorials/
My Youtube Channel: https://www.youtube.com/user/MorvanZhou
More about Reinforcement learning: https://morvanzhou.github.io/tutorials/machine-learning/reinforcement-learning/
Dependencies:
torch: 0.4
gym: 0.8.1
numpy
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym

# Hyper Parameters
BATCH_SIZE = 32
LR = 0.01                   # learning rate
EPSILON = 0.9               # greedy policy
GAMMA = 0.9                 # reward discount
TARGET_REPLACE_ITER = 100   # target update frequency
MEMORY_CAPACITY = 2000
env = gym.make('CartPole-v0')
env = env.unwrapped
N_ACTIONS = env.action_space.n
N_STATES = env.observation_space.shape[0]
ENV_A_SHAPE = 0 if isinstance(env.action_space.sample(), int) else env.action_space.sample().shape     # to confirm the shape


class Net(nn.Module):
    def __init__(self, ):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(N_STATES, 50)
        self.fc1.weight.data.normal_(0, 0.1)   # initialization
        self.out = nn.Linear(50, N_ACTIONS)
        self.out.weight.data.normal_(0, 0.1)   # initialization

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        actions_value = self.out(x)
        return actions_value


class DQN(object):
    def __init__(self):
        self.eval_net, self.target_net = Net(), Net()

        self.learn_step_counter = 0                                     # for target updating
        self.memory_counter = 0                                         # for storing memory
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES * 2 + 2))     # initialize memory
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)
        self.loss_func = nn.MSELoss()

    def choose_action(self, x):
        x = torch.unsqueeze(torch.FloatTensor(x), 0)
        # input only one sample
        if np.random.uniform() < EPSILON:   # greedy
            actions_value = self.eval_net.forward(x)
            action = torch.max(actions_value, 1)[1].data.numpy()
            action = action[0] if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)  # return the argmax index
        else:   # random
            action = np.random.randint(0, N_ACTIONS)
            action = action if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)
        return action

    def store_transition(self, s, a, r, s_):
        transition = np.hstack((s, [a, r], s_))
        # replace the old memory with new memory
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index, :] = transition
        self.memory_counter += 1

    def learn(self):
        # target parameter update
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
        self.learn_step_counter += 1

        # sample batch transitions
        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
        b_memory = self.memory[sample_index, :]
        b_s = torch.FloatTensor(b_memory[:, :N_STATES])
        b_a = torch.LongTensor(b_memory[:, N_STATES:N_STATES+1].astype(int))
        b_r = torch.FloatTensor(b_memory[:, N_STATES+1:N_STATES+2])
        b_s_ = torch.FloatTensor(b_memory[:, -N_STATES:])

        import ipdb; ipdb.set_trace()
        
        # q_eval w.r.t the action in experience
        q_eval = self.eval_net(b_s).gather(1, b_a)  # shape (batch, 1)
        q_next = self.target_net(b_s_).detach()     # detach from graph, don't backpropagate
        q_target = b_r + GAMMA * q_next.max(1)[0].view(BATCH_SIZE, 1)   # shape (batch, 1)
        loss = self.loss_func(q_eval, q_target)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

dqn = DQN()

print('\nCollecting experience...')
for i_episode in range(400):
    s = env.reset()
    ep_r = 0
    while True:
        env.render()
        a = dqn.choose_action(s)

        # take action
        s_, r, done, info = env.step(a)

        # modify the reward
        x, x_dot, theta, theta_dot = s_
        r1 = (env.x_threshold - abs(x)) / env.x_threshold - 0.8
        r2 = (env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians - 0.5
        r = r1 + r2

        dqn.store_transition(s, a, r, s_)

        ep_r += r
        if dqn.memory_counter > MEMORY_CAPACITY:
            dqn.learn()
            if done:
                print('Ep: ', i_episode,
                      '| Ep_r: ', round(ep_r, 2))

        if done:
            break
        s = s_



Now, let's take a look at our learned policy in action:

In [ ]:
env = make_env("CartPole-v1", monitor=True)
obs = env.reset()
while True:
    obs = torch.FloatTensor(obs).unsqueeze(0)
    act = agent.pick_action(obs, force_greedy=True)
    obs, rew, done, info = env.step(act)
    if done: break
env.close()

HTML(data=get_html_video_string(env.file_infix))

In [ ]:
def plot_progress(data, name=None):
    name = name or "Mean reward"
    plt.plot(data['misc/timestep'], data['reward/reward_mean'], label=name)
    lower = np.array(data['reward/reward_mean'] - data['reward/reward_std']).clip(-1e5, +1e5)
    upper = np.array(data['reward/reward_mean'] + data['reward/reward_std']).clip(-1e5, +1e5)
    examples = np.array(data['misc/timestep'])
    plt.fill_between(list(examples), list(lower), list(upper), color='blue', alpha=0.1)
    plt.legend(loc='best')
    plt.xlabel("Timestep")
    plt.ylabel("Reward")

The following cell plots a mean reward against training iterations. Note, that the agent may sometimes degrade its performance.

In [ ]:
plt.figure(dpi=300)
df = pd.read_csv(str(log_root / "cartpole-v1" / "logs_dqn.csv"), sep=";")
plot_progress(df)

# LunarLander

Okay, now that we're done with very simple CartPole environment, let's move to something more complicated. 

LunarLander is a simulation game where a player has to control the capsule and land it on the zone marked with flags. If an agent achieves more than 200 scores, the environment is said to be solved.

##### First, you should install Box2D support in Gym: 

In [ ]:
!pip install 'gym[box2d]'

##### The random policy quickly fails the task:

In [ ]:
env = make_env("LunarLander-v2", monitor=True)
env.reset()
while True:
    action = env.action_space.sample()
    if env.step(action)[2]: break
env.close()

HTML(data=get_html_video_string(env.file_infix))

#### Task 3: Play a bit with hyperparameters. You have to gain some intuition about what each of them does. You have to achieve mean score of 200 in ~200 000 iterations

In [ ]:
env = make_env("LunarLander-v2", SEED)

value_network = get_network("mlp")(
    env.observation_space.shape[0],
    ...,
    env.action_space.n)

agent = DQN(
    value_network,
    env.action_space,
    eps=...,
    eps_decay=...,
    batch_size=...,
    learning_rate=...,
    discount_factor=...,
)

In [ ]:
agent = train(
    agent,
    env,
    log_dir=log_root / "lunarlander-v2",
    prefix="_dqn",
    buffer_size=...,
    n_steps=2*10**5,
    warmup_steps=...,
    target_update_every=...,
    log_every=4000,
    save_every=1000
)

In [ ]:
env = make_env("LunarLander-v2", monitor=True)
obs = env.reset()
while True:
    obs = torch.FloatTensor(obs).unsqueeze(0)
    act = agent.pick_action(obs, force_greedy=True)
    obs, rew, done, info = env.step(act)
    if done: break
env.close()

HTML(data=get_html_video_string(env.file_infix))

In [ ]:
plt.figure(dpi=300)
df = pd.read_csv(str(log_root / "lunarlander-v2" / "logs_dqn.csv"), sep=";")
plot_progress(df)

## Double DQN

One of the main drawbacks of vanila DQN is that it overestimates Q-values. More details could be found in the original paper. In a couple of words, we have to estimate an expecation of maximum, not a maximum of expectations as we do in vanila DQN. The trick is to use the second network for the updates (we use target network for this):

$$y = r + \gamma Q_{target}(s^\prime, \text{argmax}_a Q(s^\prime, a) )$$

Compared to 

$$y = r + \gamma Q_{target}(s^\prime, \text{argmax}_a Q_{target}(s^\prime, a) )$$

from vanila DQN. Which is the same as:

$$y = r + \gamma \text{max}_a Q_{target}(s^\prime, a)$$

Double Q-learning paper: https://papers.nips.cc/paper/3964-double-q-learning <br>
Double DQN paper: https://arxiv.org/abs/1509.06461

#### Task 4. Implement Double DQN in `lib/agent.py`. 

 Now, let's try it for the LunarLander environment:

In [ ]:
env = make_env("LunarLander-v2", SEED)

value_network = get_network("mlp")(
    env.observation_space.shape[0],
    ...,
    env.action_space.n)

agent = DQN(
    value_network,
    env.action_space,
    eps=...,
    eps_decay=...,
    batch_size=...,
    learning_rate=...,
    discount_factor=...,
    double=True
)

In [ ]:
agent = train(
    agent,
    env,
    log_dir=log_root / "lunarlander-v2",
    prefix="_double_dqn",
    buffer_size=...,
    n_steps=2*10**5,
    warmup_steps=...,
    target_update_every=...,
    log_every=4000,
    save_every=1000
)

### You should obtain better curve for Double DQN, however it is not always the case. For such environent the improvement could be neglectable. Although, you have to at least reach the same performance. 
### You may want to compare the performance on _CartPole_ as, unlike _LunarLander_ , its reward function is dense. However, it may be too simple to reveal any difference.
### Thus, it is much better to compare DQN and Double DQN on Atari benchmark.

In [ ]:
plt.figure(dpi=300)
df = pd.read_csv(str(log_root / "lunarlander-v2" / "logs_dqn.csv"), sep=";")
df_double = pd.read_csv(str(log_root / "lunarlander-v2" / "logs_double_dqn.csv"), sep=";")
plot_progress(df, "DQN")
plot_progress(df_double, "Double DQN")

### Look at the graphs of estimated Q values by DQN and Double DQN. Could you describe what's happening?

In [ ]:
plt.figure(dpi=300)
plt.xlabel("Timestep")
plt.ylabel("estimated Q value")
plt.plot(df['misc/timestep'], df['misc/q_est'], label="DQN")
plt.plot(df_double['misc/timestep'], df_double['misc/q_est'], label="Double DQN")
plt.legend(loc="best");

# ATARI [Optional]

In this section, you can try the algorithm on the larger problem from Atari suit. This section is optional. The training procedure is computationally cumbersome, so that ideally one may want to use GPU and powerful CPU for this task. Also note, that usually people choose larger `buffer_size` for this benchmark, which may consume anormous amount of RAM. To avoid the memory overflow, you may consider to use lossless compression algorithm to save a lot of space by compressing observations. See RLLib or Catalyst.RL for example.

If you decided to go through the task, note, that typical choice of hyperparameters for Atari differs from previous ones. See RLLib's configs, for example, as a reference.

In [ ]:
!pip install 'gym[atari]'

#### Clone and install `baselines` lib. We'll use atari wrappers from it.

Here are commands for that:
```bash
git clone https://github.com/openai/baselines
cd baselines
pip install -e .
```

In [ ]:
from baselines.common.atari_wrappers import make_atari, wrap_deepmind
from lib.utils import TransposeImage

In [ ]:
def make_env(name, seed=None, monitor=False):
    env = gym.make(name)

    is_atari = hasattr(gym.envs, 'atari') and isinstance(
        env.unwrapped, gym.envs.atari.atari_env.AtariEnv)

    if is_atari:
        max_epi_steps = 108000
        if max_epi_steps < 0:
            max_epi_steps = None

        env = make_atari(name, max_epi_steps)
        env = wrap_deepmind(env, True, True, True, True)
        env = TransposeImage(env, op=[2, 0, 1])
        
    if monitor:
        env = wrappers.Monitor(env, "./gym-results", force=True)
    if seed:
        env.seed(seed)


    return env

In [ ]:
env = make_env("BreakoutNoFrameskip-v4", monitor=True)
obs = env.reset()
while True:
    act = env.action_space.sample()
    if env.step(act)[2]: break
env.close()

HTML(data=get_html_video_string(env.file_infix))

In [ ]:
env = make_env("BreakoutNoFrameskip-v4", SEED)

value_network = get_network("vision")(
    env.observation_space.shape[0],
    256,
    env.action_space.n)

agent = DQN(
    value_network,
    env.action_space,
    eps=...,
    eps_decay=...,
    batch_size=...,
    learning_rate=...,
    discount_factor=...,
    double=...,
)

In [ ]:
agent = train(
    agent,
    env,
    log_dir=log_root / "breakout-v4",
    prefix="_dqn",
    buffer_size=...,
    n_steps=...,
    warmup_steps=...,
    target_update_every=...,
    log_every=...,
    save_every=...,
)

In [ ]:
env = make_env("BreakoutNoFrameskip-v4", monitor=True)
obs = env.reset()
while True:
    obs = torch.FloatTensor(obs).unsqueeze(0)
    act = agent.pick_action(obs, force_greedy=True)
    obs, rew, done, info = env.step(act)
    if done: break
env.close()

HTML(data=get_html_video_string(env.file_infix))

In [ ]:
plt.figure(dpi=300)
df = pd.read_csv(str(log_root / "breakout-v4" / "logs_dqn.csv"), sep=";")
plot_progress(df)